<a href="https://colab.research.google.com/github/DunkleCat/a3c-tensorflow/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Header


## Libraries


In [1]:
!pip install tf-agents

import csv

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tf_agents.utils import value_ops

import gym
import numpy as np
from threading import Thread, Lock
from multiprocessing import cpu_count

tf.keras.backend.set_floatx('float64')

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

     |████████████████████████████████| 1.1MB 8.3MB/s 
Found GPU at: /device:GPU:0


## Global variables

In [2]:
def getNumberOfWorkers():
  return cpu_count()
  # return 1

def getMaxEpisodes():
  return 1000

def getBatchSize():
  return 10

def env_name():
  return "CartPole-v1"

def getTypology():
  return ("classic",)
  # Options:
  # ("classic",)
  # ("atari", "visual")
  # ("atari", "ram")

CUR_EPISODE = 0

# Random Agent


In [3]:
import gym
env = gym.make(env_name())
for i_episode in range(20):
  observation = env.reset()
  episode_reward, done = 0, False
  while not done:
    # env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    episode_reward += reward
  print("Episode finished with a reward of {}".format(episode_reward))
  episode_reward = 0
env.close()

Episode finished with a reward of 15.0
Episode finished with a reward of 46.0
Episode finished with a reward of 15.0
Episode finished with a reward of 16.0
Episode finished with a reward of 24.0
Episode finished with a reward of 20.0
Episode finished with a reward of 8.0
Episode finished with a reward of 18.0
Episode finished with a reward of 22.0
Episode finished with a reward of 16.0
Episode finished with a reward of 12.0
Episode finished with a reward of 15.0
Episode finished with a reward of 15.0
Episode finished with a reward of 38.0
Episode finished with a reward of 13.0
Episode finished with a reward of 18.0
Episode finished with a reward of 23.0
Episode finished with a reward of 23.0
Episode finished with a reward of 10.0
Episode finished with a reward of 23.0


## Main

In [4]:
def main():
  agent = Agent()
  agent.train()

# ActorCritic


## Help functions

In [5]:
def get_input_classic(input, single):
  
  if not single:
    return input
  else:
    shape = input.shape
    n_samples = 1
    new_input = np.ndarray((n_samples,
                            shape[0]))
    new_input[0] = input
    return new_input

def get_input_atari(input, single):

  if single:
    shape = input.shape
    n_samples = 1
  else:
    shape = input[0].shape
    n_samples = len(input)

  if getTypology()[1] is "visual":
    new_input = np.ndarray((n_samples,
                            1,
                            shape[0],
                            shape[1],
                            shape[2]))
  elif getTypology()[1] is "ram":
    new_input = np.ndarray((n_samples,
                            1,
                            shape[0]))

  if single:
      new_input[0][0] = input
  else:
    for k in range(len(input)):
        new_input[k][0] = input[k]

  return new_input

def init_state_shape(state_shape):
  if getTypology()[0] is "classic":
    return state_shape
  elif getTypology()[1] is "visual":
      return np.ndarray(shape = (1,
                                 state_shape[0],
                                 state_shape[1],
                                 state_shape[2])).shape
  elif getTypology()[1] is "ram":
    return np.ndarray(shape = (1,
                               state_shape[0])).shape

## Actor

In [6]:
class Actor:

  def __init__(self, state_shape, action_shape):
    self.state_shape = init_state_shape(state_shape)    
    self.action_shape = action_shape
    self.model = create_model(self.state_shape, self.action_shape)
    self.opt = tf.keras.optimizers.Adam(0.0005)

  def get_action(self, input):
    input = eval("get_input_" + getTypology()[0] + "(input, single = True)")
    action_distribution = self.model.predict(input)
    return np.random.choice(self.action_shape, p = action_distribution[0])
    # return np.argmax(action_distribution == action)

  def train(self, input, actions, advantages):

    def compute_loss(actions, action_dist, advantages):
      # Compute policy loss
      scc = tf.keras.losses.SparseCategoricalCrossentropy()
      # print(advantages)
      policy_loss = scc(actions, action_dist, sample_weight = tf.stop_gradient(advantages))
      # tmp = tf.one_hot(actions,len(action_dist[0]),dtype=tf.float64)
      # tmp = tf.reduce_sum(tf.math.multiply(tmp, action_dist))
      # tmp = tf.math.log(tmp)
      # policy_loss = tf.reduce_sum(tmp * advantages)
      # Compute entropy
      cc = tf.keras.losses.CategoricalCrossentropy()
      entropy = 0.01 * cc(action_dist, action_dist)
      # entropy = 0.001 * tf.reduce_sum(action_dist * tf.math.log(action_dist))
      return policy_loss - entropy

    with tf.GradientTape() as tape:
      input = eval("get_input_" + getTypology()[0] + "(input, single = False)")
      # input = get_input(input)
      action_dist = self.model(input, training = True)
      loss = compute_loss(actions, action_dist, advantages)
    grads = tape.gradient(loss, self.model.trainable_variables)
    self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
    return loss

## Critic

In [7]:
class Critic:
  def __init__(self, state_shape):
    self.state_shape = init_state_shape(state_shape)
    self.model = create_model(self.state_shape, 1)
    self.opt = tf.keras.optimizers.Adam(0.001)

  def get_value(self, input, single):
    input = eval("get_input_" + getTypology()[0] + "(input, single = single)")
    return self.model.predict(input)

  def train(self, input, td_targets):

    def compute_loss(v_pred, v_targets):
      mse = tf.keras.losses.MeanSquaredError()
      return mse(v_pred, v_targets)

    with tf.GradientTape() as tape:
      input = eval("get_input_" + getTypology()[0] + "(input, single = False)")
      v_pred = self.model(input, training=True)
      assert v_pred.shape == td_targets.shape
      loss = compute_loss(v_pred, tf.stop_gradient(td_targets))
    grads = tape.gradient(loss, self.model.trainable_variables)
    self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
    return loss

## Model

In [8]:
def create_model(input_shape, output_shape):
  
  if output_shape is 1: #Critic
    activation_function = "relu"
  else: # Actor
    activation_function = "softmax"

  function_name = "create_model_" + getTypology()[0]
  function_arguments = "(input_shape, output_shape, activation_function)"
  return eval(function_name + function_arguments)

def create_model_atari(input_shape, output_shape, activation_function):
  if getTypology()[1] is "visual":
    return keras.Sequential(
        [
         layers.InputLayer(input_shape),
         layers.ConvLSTM2D(16,4,2, 
                           data_format='channels_last',
                           dropout = 0.5,
                           #recurrent_dropout = 0.5,
                           return_sequences = True),
         layers.BatchNormalization(),
         # layers.ConvLSTM2D(32,4,1, 
         #                   data_format='channels_last',
         #                   dropout = 0.5,
         #                   recurrent_dropout = 0.5,
         #                   return_sequences = True),
         # layers.BatchNormalization(),
         layers.ConvLSTM2D(32,4,1, 
                           data_format='channels_last',
                           dropout = 0.5,
                           #recurrent_dropout = 0.5,
                           return_sequences = True),
         layers.BatchNormalization(),
         layers.ConvLSTM2D(1,1,1, 
                           data_format='channels_last',
                           dropout = 0.5,
                           recurrent_dropout = 0.5,
                           return_sequences = False),
         layers.BatchNormalization(),
         layers.AveragePooling2D(),
         layers.Flatten(),
         layers.Dropout(0.5),
         layers.Dense(256,
                      activation="relu"),
         # layers.Dense(128,
         #              activation="relu"),
         # layers.Dense(64,
         #              activation="relu"),
         layers.Dense(output_shape, 
                      activation = activation_function)
        ]
    )
  elif getTypology()[1] is "ram":
    return keras.Sequential(
        [
         layers.InputLayer(input_shape),
         layers.LSTM(32,
                     dropout=0.2),
         layers.BatchNormalization(),
         layers.Dropout(0.2),
         layers.Dense(32,
                      activation="relu"
                      ),
         layers.Dropout(0.2),
         layers.Dense(64,
                      activation="relu"),
         layers.Dropout(0.2),
         layers.Dense(32,
                      activation="relu"),
         layers.Dense(output_shape,
                      activation = activation_function)       
        ]
    )

def create_model_classic(input_shape, output_shape, activation_function):
  return keras.Sequential(
      [
       layers.Input(input_shape),
       layers.Dropout(0.2),
       layers.Dense(32, activation = "relu"),
       layers.BatchNormalization(),
       layers.Dropout(0.2),
       layers.Dense(16, activation = "relu"),
       layers.Dense(16, activation = "relu"),
       layers.Dense(output_shape,
                    activation = activation_function)
      ]
  )

# Agent

In [9]:
class Agent:
    def __init__(self):
        env = gym.make(env_name())
        self.state_shape = env.observation_space.shape
        self.action_shape = env.action_space.n
        env.close()
        
        self.global_actor = Actor(self.state_shape, self.action_shape)
        self.global_critic = Critic(self.state_shape)

    def train(self):
        workers = []

        for i in range(getNumberOfWorkers()):
            workers.append(WorkerAgent(self.global_actor,
                                       self.global_critic))
        for worker in workers:
            worker.start()

        for worker in workers:
            worker.join()

# WorkerAgent

In [10]:
class WorkerAgent(Thread):
    def __init__(self, global_actor, global_critic):
        Thread.__init__(self)
        self.lock = Lock()
        self.env = gym.make(env_name())
        # print(env.observation_space.shape)
        # print(env.observation_space.shape[0])
        self.state_shape = env.observation_space.shape
        self.action_shape = env.action_space.n

        self.global_actor = global_actor
        self.global_critic = global_critic
        self.actor = Actor(self.state_shape, self.action_shape)
        self.critic = Critic(self.state_shape)

        self.actor.model.set_weights(self.global_actor.model.get_weights())
        self.critic.model.set_weights(self.global_critic.model.get_weights())

    def train(self):

        def list_to_batch(list):
            batch = list[0]

            for elem in list[1:]:
                batch = np.append(batch, elem, axis = 0)
            return batch

        def n_step_td_target(rewards, next_v_value, done):
            td_targets = np.zeros_like(rewards)
            cumulative = 0.99 * next_v_value if not done else 0

            for k in reversed(range(0, len(rewards))):
            # for k in range(0, len(rewards)):
                cumulative = pow(0.99, k) * cumulative + rewards[k]
                td_targets[k] = cumulative
            return td_targets

        global CUR_EPISODE

        while getMaxEpisodes() >= CUR_EPISODE:
            state_batch = []
            action_batch = []
            reward_batch = []
            episode_reward, episode_actor_loss, episode_critic_loss, done = 0, 0, 0, False

            state = self.env.reset()

            while not done:
                # self.env.render()
                action = self.actor.get_action(state)
                next_state, reward, done, _ = self.env.step(action) 
                episode_reward += reward

                if episode_reward >= 200:
                  done = True
                # print(reward)
                # print(episode_reward)

                # action = np.reshape(action, [1, 1])
                # next_state = np.reshape(next_state, [1, self.state_shape])
                # reward = reward + tf.math.reduce_sum(reward_batch) * pow(0.99, len(state_batch))
                reward = np.reshape(reward, [1, 1])

                state_batch.append(state)
                action_batch.append(action)
                reward_batch.append(reward)

                if len(state_batch) >= getBatchSize() or done:
                    actions = np.array(action_batch)
                    # state_batch.append(next_state)
                    states = np.array(state_batch)
                    # actions = list_to_batch(action_batch)
                    # states = list_to_batch(state_batch)
                    rewards = list_to_batch(reward_batch)

                    # td_targets = n_step_td_target(rewards, next_v_value, done)
                    
                    values = self.critic.get_value(states, single = False)
                    final_value = self.critic.get_value(next_state, single = True)
                    discounts = 0.99 * np.ones(values.shape)
                    # print(values.shape)
                    # print(final_value[0])
                    # print(final_value[0].shape)
                    # print(discounts.shape)
                    # print(rewards.shape)
                    # print("LOL\n\n")
                    advantages = value_ops.generalized_advantage_estimation(values,
                                                                            final_value[0],
                                                                            discounts, 
                                                                            rewards)
                    # advantages = tf.keras.utils.normalize(advantages)
                    # print(advantages.shape)
                    # advantages = rewards + pow(0.99, len(state_batch)+1) * values[1:] - values[:-1])
                    # states = states[:-1]
                    
                    # print(td_targets)
                    # print(advantages)
                    with self.lock:
                        # print(states.shape)
                        # print(actions.shape)
                        # print(advantages.shape)
                        actor_loss = self.global_actor.train(states, actions, advantages)
                        self.actor.model.set_weights(self.global_actor.model.get_weights())

                        critic_loss = self.global_critic.train(states, n_step_td_target(rewards, final_value, done))
                        self.critic.model.set_weights(self.global_critic.model.get_weights())
                        
                        episode_actor_loss += actor_loss
                        episode_critic_loss += critic_loss
        
                    state_batch = []
                    action_batch = []
                    reward_batch = []

                state = next_state

            if CUR_EPISODE%5 is 0:
              print("EP{}: Reward = {}, Actor Loss = {}, Critic Loss = {}".format(CUR_EPISODE, 
                                                                                  episode_reward, 
                                                                                  episode_actor_loss, 
                                                                                  episode_critic_loss))
              self.actor.model.save("/content/drive/My Drive/Machine Learning/models/" + str(env_name()) + "_actor")
              self.critic.model.save("/content/drive/My Drive/Machine Learning/models/" + str(env_name()) + "_critic")
              
              with open("/content/drive/My Drive/Machine Learning/logs/" + str(env_name()) + "_loss_actor", "w") as csv_file:
                csv_writer = csv.writer(csv_file, delimiter=',')
                csv_writer.writerow([CUR_EPISODE, episode_actor_loss])

              with open("/content/drive/My Drive/Machine Learning/logs/" + str(env_name()) + "_loss_critic", "w") as csv_file:
                csv_writer = csv.writer(csv_file, delimiter=',')
                csv_writer.writerow([CUR_EPISODE, episode_critic_loss])

              with open("/content/drive/My Drive/Machine Learning/logs/" + str(env_name()) + "_reward", "w") as csv_file:
                csv_writer = csv.writer(csv_file, delimiter=',')
                csv_writer.writerow([CUR_EPISODE, episode_reward])
              
            CUR_EPISODE += 1

    def run(self):
        self.train()

# Entrypoint

In [11]:
if __name__ == "__main__":
  main()
else:
  main()

# Load model

EP0: Reward = 11.0, Actor Loss = 4.577521560192109, Critic Loss = 28.675021469593048
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
EP0: Reward = 30.0, Actor Loss = 12.140057108402253, Critic Loss = 86.16731643676758
INFO:tensorflow:Assets written to: /content/drive/My Drive/Machine Learning/models/CartPole-v1_actor/assets
INFO:tensorflow:Assets written to: /content/drive/My Drive/Machine Learning/models/CartPole-v1_actor/assets
INFO:tensorflow:Assets written to: /content/drive/My Drive/Machine Learning/models/CartPole-v1_critic/assets
INFO:tensorflow:Assets written to: /content/drive/My Drive/Machine Learning/models/CartPole-v1_critic/assets
EP5: Reward = 14.0, Actor Loss = 5.5110784590244295, Critic Loss = 31.14291000366211
EP5: Reward = 32.0, Actor Loss = 12.297971955537797, Critic Loss = 81.5181280

KeyboardInterrupt: ignored